In [42]:
import gradio as gr
from openai import OpenAI
import json



In [54]:
openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

In [44]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [45]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [46]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [47]:
tools = [{"type" : "function" ,"function" : price_function}]

In [48]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    print(f"Tool call: {tool_call.function.name} with arguments {tool_call.function.arguments}")
    arguments = json.loads(tool_call.function.arguments)
    print(f"Tool call arguments: {arguments}")
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-oss", messages=messages, tools=tools)
    print(response)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        print(messages)
        response = openai.chat.completions.create(model="gpt-oss", messages=messages)
    final_text = response.choices[0].message.content
    # if "<think>" in final_text:
    #     final_text = final_text.split("</think>")[-1].strip()
    return  final_text





# def chat(message, history):
#     messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    
#     response = openai.chat.completions.create(
#         model="gpt-oss",
#         messages=messages
#     )
    
#     final_text = response.choices[0].message.content
    
#     # Remove <think> ... </think> part if present
#     if "<think>" in final_text:
#         final_text = final_text.split("</think>")[-1].strip()
    
#     return final_text

In [61]:
gr.ChatInterface(fn=chat , type="messages").launch()

* Running on local URL:  http://127.0.0.1:7875
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "C:\Users\dragk\AppData\Roaming\Python\Python313\site-packages\gradio\queueing.py", line 626, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<5 lines>...
    )
    ^
  File "C:\Users\dragk\AppData\Roaming\Python\Python313\site-packages\gradio\route_utils.py", line 350, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<11 lines>...
    )
    ^
  File "C:\Users\dragk\AppData\Roaming\Python\Python313\site-packages\gradio\blocks.py", line 2250, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<8 lines>...
    )
    ^
  File "C:\Users\dragk\AppData\Roaming\Python\Python313\site-packages\gradio\blocks.py", line 1755, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\dragk\AppD